In [2]:
from bvhtoolbox import Bvh

In [3]:
bvh_filepath = '../samples/beatsaber_0_1_b.bvh'

with open(bvh_filepath) as f:
    bvh_file = Bvh(f.read())

In [4]:
bvh_file.get_joints_names()

['Hips',
 'LeftUpperLeg',
 'LeftLowerLeg',
 'LeftFoot',
 'LeftToes',
 'RightUpperLeg',
 'RightLowerLeg',
 'RightFoot',
 'RightToes',
 'Spine',
 'Chest',
 'UpperChest',
 'LeftShoulder',
 'LeftUpperArm',
 'LeftLowerArm',
 'LeftHand',
 'Neck',
 'Head',
 'RightShoulder',
 'RightUpperArm',
 'RightLowerArm',
 'RightHand']

In [7]:
root_joint = bvh_file.get_joint('Hips')
for j in root_joint:
    print(j)


OFFSET 0.00 0.00 0.00
CHANNELS 6 Xposition Yposition Zposition Zrotation Xrotation Yrotation
JOINT LeftUpperLeg
JOINT RightUpperLeg
JOINT Spine


In [79]:
def print_values(iteratable_obj):
    for o in iteratable_obj:
        print(o)

In [80]:
root_joint = bvh_file.get_joint(bvh_file.root['ROOT'][0])
print_values(root_joint)

OFFSET 0.00 0.00 0.00
CHANNELS 6 Xposition Yposition Zposition Zrotation Xrotation Yrotation
JOINT LeftUpperLeg
JOINT RightUpperLeg
JOINT Spine


In [81]:
print(bvh_file.frames[0])

['0.308680', '1.039460', '-0.657690', '-6.499474', '3.104017', '19.372750', '0.288538', '-13.627510', '-0.481895', '1.415104', '27.674240', '-0.348695', '5.987649', '-17.031240', '17.851630', '0.000005', '0.015295', '0.210898', '-10.498260', '5.079259', '-32.282910', '0.004505', '0.088981', '-0.000413', '8.421024', '13.075360', '-4.449048', '-0.000220', '-5.874985', '0.000417', '-0.807257', '0.418094', '0.379163', '0.000000', '0.000000', '0.000000', '-1.619897', '0.831004', '0.764855', '-1.123447', '0.243423', '20.095910', '-74.432230', '5.420115', '3.616205', '-2.517669', '9.464352', '-16.945360', '42.967510', '25.716490', '-28.569420', '10.282900', '-1.663791', '-6.931583', '9.912028', '-1.905848', '-6.885983', '-1.028874', '2.234648', '-8.656346', '80.952170', '-13.740250', '35.634900', '16.142320', '11.220650', '69.696740', '-40.467810', '-16.410430', '12.718190']


In [45]:
def get_bindpose_table(bvh_in:Bvh):
    root_joint = bvh_in.get_joint(bvh_in.root['ROOT'][0])

get_bindpose_table(bvh_file)

ROOT Hips


In [54]:
from collections import OrderedDict
import transforms3d as t3d
from transforms3d import quaternions as t3dq

class BvhSkeltonNode:
    def __init__(self, name, offset):
        self.children = []
        self.parent:BvhSkeltonNode = None
        
        self.name = name
        self.local_position = offset,
        self.local_rotation = None,
        self.local_rotation_q = [0,0,0,0]
        self.local_transform = None

        self.world_position = None
        self.world_rotation = None
        self.world_rotation_q = None
        self.world_transform = None

    def calculate_world_transform(self):
        self.matLocal = t3d.affines.compose(self.local_position, self.local_rotation, [1,1,1])

        if self.parent != None:
            self.world_transform = self.parent.world_transform.dot(self.local_transform)
        else:
            self.world_transform = self.local_transform

        self.world_position, self.world_rotation, _, _ = t3d.affines.decompose(self.world_transform)
        self.world_rotation_quat = t3dq.mat2quat(self.world_rotation)

        for child in self.children:
            child.CalculateWorld()

class BvhSkeleton:
    def __init__(self, bvh_filepath):
        self.bvh_filepath = bvh_filepath
        self.root = None
        self.nodes = OrderedDict()
        self.nodes_flat = []

In [112]:
from math import radians
def ZXYtoQUAD_Unity(z, x, y, y_up = True):
    qZ = t3dq.axangle2quat([0,0,1], radians(z), True) #FORWARD/Z
    qX = t3dq.axangle2quat([1,0,0], radians(x), True) #RIGHT/X
    qY = t3dq.axangle2quat([0,1,0], radians(y), True) #UP/Y
    q = t3dq.qmult(t3dq.qmult(qZ , qX) , qY)
    #q /= t3dq.qnorm(q)

    if y_up:
        return q * [1,1,-1,-1]
    
    return q * [1,1,1,-1]

def arrZXYtoQUAD_Unity(euler_in, y_up = True):
    return ZXYtoQUAD_Unity(euler_in[0], euler_in[1], euler_in[2], y_up)

euler = [-6.499474, 3.104017, 19.372750]
print(arrZXYtoQUAD_Unity(euler))

[ 0.98405566  0.03618986 -0.16640931  0.05130978]


In [84]:
bvh_file.root['ROOT']

['Hips']

In [114]:
import pandas as pd
import numpy as np

def BvhToTrainSet(bvh_filepath):
    with open(bvh_filepath) as f:
        bvh_file = Bvh(f.read())

    #Bvh Information
    name_rootNode = bvh_file.root['ROOT'][0]
    name_nodesFlat = bvh_file.get_joints_names()
    
    #Output Names
    name_outputs = list()
    name_outputs.append(f'{name_rootNode}_px')
    name_outputs.append(f'{name_rootNode}_py')
    name_outputs.append(f'{name_rootNode}_pz')
    for n in name_nodesFlat:
        name_outputs.append(f'{n}_rw')
        name_outputs.append(f'{n}_rx')
        name_outputs.append(f'{n}_ry')
        name_outputs.append(f'{n}_rz')

    #Pandas Dataframe
    feature_count = len(name_outputs)
    frame_count = len(bvh_file.frames)

    #Frame data
    np_frame = np.empty((frame_count ,feature_count))
    for fn, f in enumerate(bvh_file.frames):
        np_frame[fn,0] = f[0]
        np_frame[fn,1] = f[1]
        np_frame[fn,2] = f[2]
        for r_idx in range(len(name_nodesFlat)):
            f_idx = 3 + r_idx * 3
            out_idx = 3 + (r_idx * 4)
            r_q = ZXYtoQUAD_Unity(float(f[f_idx]), float(f[f_idx+1]), float(f[f_idx+2]))
            np_frame[fn,out_idx] = r_q[0]      
            np_frame[fn,out_idx+1] = r_q[1]      
            np_frame[fn,out_idx+2] = r_q[2]      
            np_frame[fn,out_idx+3] = r_q[3]         

    return pd.DataFrame(data=np_frame, columns=name_outputs)

df_out = BvhToTrainSet('../samples/beatsaber_0_1_b.bvh')
df_out.head()

,Hips_px,Hips_py,Hips_pz,Hips_rw,Hips_rx,Hips_ry,Hips_rz,LeftUpperLeg_rw,LeftUpperLeg_rx,LeftUpperLeg_ry,...,RightUpperArm_ry,RightUpperArm_rz,RightLowerArm_rw,RightLowerArm_rx,RightLowerArm_ry,RightLowerArm_rz,RightHand_rw,RightHand_rx,RightHand_ry,RightHand_rz
0,0.30868,1.03946,-0.65769,0.984056,0.03619,-0.166409,0.05131,0.992924,-0.118630,0.004474,...,-0.157161,-0.585716,0.800803,-0.000407,-0.574301,-0.169981,0.917502,-0.095173,-0.151917,0.355039
1,0.30670,1.04040,-0.66078,0.981641,0.04235,-0.179760,0.04768,0.992139,-0.124355,0.013971,...,-0.152640,-0.591517,0.806116,0.005106,-0.569579,-0.160407,0.915394,-0.088945,-0.160569,0.358273
2,0.30528,1.04105,-0.66405,0.979352,0.04824,-0.191140,0.04482,0.991256,-0.130115,0.021335,...,-0.155632,-0.597815,0.815994,0.006491,-0.560084,-0.142893,0.913491,-0.082485,-0.168685,0.360937
3,0.30528,1.04105,-0.66405,0.979352,0.04824,-0.191140,0.04482,0.991256,-0.130115,0.021335,...,-0.155632,-0.597815,0.815994,0.006491,-0.560084,-0.142893,0.913491,-0.082485,-0.168685,0.360937
4,0.30336,1.04144,-0.66707,0.976948,0.05444,-0.201770,0.04357,0.990128,-0.137180,0.027729,...,-0.162031,-0.602416,0.830559,0.002589,-0.542156,-0.127408,0.911110,-0.083880,-0.173209,0.364474
